In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

from nicheformer.data.constants import DefaultPaths, ObsConstants, UnsConstants, VarConstants, AssayOntologyTermId, SexOntologyTermId, OrganismOntologyTermId, TissueOntologyTermId, SuspensionTypeId
from nicheformer.data.tools import qc_filter
from nicheformer.data.validate import validate
from nicheformer.data.download import download_tar
from nicheformer.data.constants import GeneExpressionOmnibus, DefaultPaths
import numpy as np
from pyensembl import EnsemblRelease

In [5]:
import anndata
import os 
import sys
import pandas as pd
import scanpy as sc
from tqdm import tqdm


In [6]:
!pwd

/root/capsule/LC_NE_analayis


In [7]:
raw_path = "/root/capsule/data/LC_NE/"
preprocessed_path = "/root/capsule/LC_NE_output/"

In [12]:
assay = str(AssayOntologyTermId.TENX_3.value) # 10x v3
organism = str(OrganismOntologyTermId.MOUSE.value) # mouse!
organism_validator = "mouse" # validaete!
tissue_type = "tissue" # or alternatively "organoid"
condition_id = "wild type" 

In [13]:
tissue = str(TissueOntologyTermId.BRAIN.value) # brain!
suspension_type = str(SuspensionTypeId.TENX_NUCLEUS.value)  # 10x nucleus!
sex_f = str(SexOntologyTermId.FEMALE.value)  # female! 
sex_m = str(SexOntologyTermId.MALE.value)   #male

In [14]:
filename = raw_path + 'adata_sc.h5ad'
filename

'/root/capsule/data/LC_NE/adata_sc.h5ad'

# load data!!!!

In [15]:
filename = raw_path + 'adata_sc.h5ad'
adata = sc.read_h5ad(filename)
genderinfo = pd.Categorical(adata.obs['sex'].replace({'M': sex_m, 'F': sex_f}))
adata.obs = adata.obs.iloc[:,:3]
adata.obs.head()

,nCount_RNA,nFeature_RNA,sample_id
AAGCCCCGTTCGTGCA-L8TX_240808_02_A05-NW-TX4122-4,82490.0,9352,AAGCCCCGTTCGTGCA-L8TX_240808_02_A05-NW-TX4122-4
AAGCTAAAGCTGGTTA-L8TX_240808_02_A05-NW-TX4122-4,54470.0,8115,AAGCTAAAGCTGGTTA-L8TX_240808_02_A05-NW-TX4122-4
AATCGAGGTTGACTGT-L8TX_240808_02_A05-NW-TX4122-4,51946.0,8066,AATCGAGGTTGACTGT-L8TX_240808_02_A05-NW-TX4122-4
AATCGCATCGATTGCA-L8TX_240808_02_A05-NW-TX4122-4,65250.0,8595,AATCGCATCGATTGCA-L8TX_240808_02_A05-NW-TX4122-4
AATGGCTCATGGTATA-L8TX_240808_02_A05-NW-TX4122-4,95637.0,9609,AATGGCTCATGGTATA-L8TX_240808_02_A05-NW-TX4122-4


In [17]:
def ensem_for_adata(adata):         
    data = EnsemblRelease(104, species='mus_musculus')    
    data.index()
    new_names = []
    all_invalid_names = []
    counter = 0
    for i,m in enumerate(adata.var_names):
        try:
            out=data.genes_by_name(m)[0].id
            new_names.append(out)
        except:# Exception as e:  
            # # print(e)
            # try:
            #     m1 = m.split(' ')[0]
            #     out=data.genes_by_name(m1)[0].id
            #     new_names.append(out)
            # except:# Exception as e:
                # print(e)
            counter += 1
            new_names.append(f'NA_{counter}')        
            all_invalid_names.append(m)
    print(f'total invalid {counter}, from {len(adata.var_names)} {100*counter/len(adata.var_names)} %')    
    adata_1 = adata.copy()    
    adata_1.var.index = (new_names)    
    adata_1 = adata_1[:,adata_1.var.index != 'NA']
    
    print(f'new data size {adata_1.shape}')
    return(adata_1,all_invalid_names)

In [19]:
adata_orig = adata.copy() # save the original 
adata_foo, all_invalid_names = ensem_for_adata(adata) 

INFO:pyensembl.download_cache:Fetching /root/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdna.all.fa.gz from URL https://ftp.ensembl.org/pub/release-104/fasta/mus_musculus/cdna/Mus_musculus.GRCm39.cdna.all.fa.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-104/fasta/mus_musculus/cdna/Mus_musculus.GRCm39.cdna.all.fa.gz to /root/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdna.all.fa.gz
INFO:pyensembl.download_cache:Fetching /root/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.ncrna.fa.gz from URL https://ftp.ensembl.org/pub/release-104/fasta/mus_musculus/ncrna/Mus_musculus.GRCm39.ncrna.fa.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-104/fasta/mus_musculus/ncrna/Mus_musculus.GRCm39.ncrna.fa.gz to /root/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.ncrna.fa.gz
INFO:pyensembl.download_cache:Fetching /root/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.pep.all.fa.gz from URL https://ftp.e

INFO:datacache.database:Inserting 842117 rows into table exon
INFO:datacache.database:Creating index on exon (seqname, start, end)
INFO:datacache.database:Creating index on exon (gene_name)
INFO:datacache.database:Creating index on exon (gene_id)
INFO:datacache.database:Creating index on exon (transcript_id)
INFO:datacache.database:Creating index on exon (transcript_name)
INFO:datacache.database:Creating index on exon (exon_id)
INFO:datacache.database:Creating index on exon (protein_id)
INFO:datacache.database:Creating index on exon (ccds_id)
INFO:datacache.database:Running sqlite query: "CREATE TABLE _datacache_metadata (version INT)"
INFO:datacache.database:Running sqlite query: "INSERT INTO _datacache_metadata VALUES (3)"
INFO:pyensembl.sequence_data:Parsing sequences from FASTA file at /root/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdna.all.fa.gz
INFO:pyensembl.sequence_data:Saving sequence dictionary to /root/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdn

total invalid 459, from 29617 1.549785596110342 %
new data size (5040, 29617)


In [22]:
adata = adata_foo.copy()

In [23]:
adata.var.keys()

Index(['vf_vst_counts_mean', 'vf_vst_counts_variance',
       'vf_vst_counts_variance.expected',
       'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable',
       'vf_vst_counts_rank', 'var.features', 'var.features.rank'],
      dtype='object')

In [24]:
adata.obs[ObsConstants.ASSAY_ONTOLOGY_TERM_ID] = pd.Categorical([assay for i in range(len(adata))])
adata.obs[ObsConstants.SEX_ONTOLOGY_TERM_ID] = genderinfo
adata.obs[ObsConstants.ORGANISM_ONTOLOGY_TERM_ID] = pd.Categorical([organism for i in range(len(adata))])
adata.obs[ObsConstants.TISSUE_ONTOLOGY_TERM_ID] = pd.Categorical([tissue for i in range(len(adata))])
adata.obs[ObsConstants.SUSPENSION_TYPE] = pd.Categorical([suspension_type for i in range(len(adata))])

# NicheFormer data schema
adata.obs[ObsConstants.CONDITION_ID] = pd.Categorical([condition_id for i in range(len(adata))])
adata.obs[ObsConstants.DONOR_ID] = adata.obs['sample_id']
adata.obs[ObsConstants.TISSUE_TYPE] = pd.Categorical([tissue_type for i in range(len(adata))])




In [26]:
adata.var[VarConstants.FEATURE_IS_FILTERED] = False

# after concatenation these are dtype=object, but need to be category
adata.obs[ObsConstants.CONDITION_ID] = adata.obs[ObsConstants.CONDITION_ID].astype('category')
adata.obs[ObsConstants.DONOR_ID] = adata.obs[ObsConstants.DONOR_ID].astype('category')

# run basic filtering with default values
print(f"\nPerforming basic quality control for this data.")
adata = qc_filter(adata=adata)



Performing basic quality control for this data.
AnnData object before filtering has 5040 cells and 29617 genes.
AnnData object after cell filtering: 5040 cells, 29617 genes.
AnnData object after gene filtering: 5040 cells, 24669 genes.


In [28]:
adata

AnnData object with n_obs × n_vars = 5040 × 24669
    obs: 'nCount_RNA', 'nFeature_RNA', 'sample_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'condition_id', 'donor_id', 'tissue_type', 'n_counts'
    var: 'vf_vst_counts_mean', 'vf_vst_counts_variance', 'vf_vst_counts_variance.expected', 'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable', 'vf_vst_counts_rank', 'var.features', 'var.features.rank', 'feature_is_filtered', 'n_cells'
    obsm: 'X_pca', 'X_umap'
    layers: 'rawcounts'

In [29]:
# adata = anndata.concat(adatas, index_unique='_')
adata.uns[UnsConstants.TITLE] = 'foo'
adata.var[VarConstants.FEATURE_IS_FILTERED] = False


In [31]:
preprocessed_path = "/root/capsule/LC_NE_output/"

In [33]:
outputfilename = 'adata_sc_preprocessed'
# run validator
print(f"\nValidating....")
adata_output, valid, errors, is_seurat_convertible = validate(adata, organism=organism_validator)

if valid:
    print(f"DONE: Successfully preprocessed this data, validation completed with status is_valid={valid}.")
    print(f"\nWRITING PREPROCESSED FILE TO: {outputfilename}.h5ad")
    adata_output.write(f"{preprocessed_path}/{outputfilename}.h5ad")
else:
    print(f"ERROR: Preprocessing of yoru LC-NE failed, validation completed with status is_valid={valid}.")


Validating....


INFO:cellxgene_schema.validate:Starting validation...
INFO:cellxgene_schema.validate:Validation complete in 0:00:02.712655 with status is_valid=True
INFO:cellxgene_schema.validate:Writing labels
INFO:cellxgene_schema.utils:enforce canonical format in X
INFO:cellxgene_schema.validate:H5AD label writing complete in 0:01:18.719670


DONE: Successfully preprocessed this data, validation completed with status is_valid=True.

WRITING PREPROCESSED FILE TO: LC_NE.h5ad


In [34]:
adata.var.index.shape

(24669,)

In [35]:
np.unique(adata.var.index).shape

(24669,)